# ❤️ Full ML Evaluation per Model

This notebook trains 10 ML models on the Heart Disease dataset and shows the following **SEPARATELY for EACH model**:

1. ✅ Classification Report (text)
2. ✅ Confusion Matrix (plot)
3. ✅ Accuracy (bar plot)
4. ✅ Loss Value & Loss Plot (if applicable)
5. ✅ AUC Score (text)
6. ✅ ROC Curve (plot)


## 📦 Step 1: Import Required Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, roc_auc_score,
    roc_curve, log_loss
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
# import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')


## 📂 Step 2: Load the Dataset

In [ ]:

df = pd.read_csv("heart.csv")
df.head()


## ⚙️ Step 3: Preprocessing

In [ ]:

X = df.drop('output', axis=1)
y = df['output']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


## 🤖 Step 4: Define ML Models

In [ ]:

models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(probability=True),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    # "LightGBM": lgb.LGBMClassifier()
}


## 📊 Step 5: Train & Evaluate Each Model Separately

In [ ]:

for name, model in models.items():
    print(f"\n================ {name} ================\n")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    loss = log_loss(y_test, y_prob)
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)

    print("📋 Classification Report:")
    print(cr)

    print(f"✅ Accuracy: {acc:.4f}")
    print(f"✅ AUC Score: {auc:.4f}")
    print(f"✅ Log Loss: {loss:.4f}")

    # 1. Confusion Matrix Plot
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Confusion Matrix - {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

    # 2. Accuracy Bar
    plt.figure(figsize=(3, 2))
    plt.bar(["Accuracy"], [acc], color='green')
    plt.ylim(0, 1)
    plt.title(f"Accuracy - {name}")
    plt.ylabel("Score")
    plt.show()

    # 3. Loss Bar
    plt.figure(figsize=(3, 2))
    plt.bar(["Log Loss"], [loss], color='red')
    plt.title(f"Log Loss - {name}")
    plt.ylabel("Loss")
    plt.show()

    # 4. ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.figure(figsize=(4, 3))
    plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {name}")
    plt.legend()
    plt.show()


## 📈 Step 6: Summary Comparison of All Models (Accuracy, AUC, Loss)

In [ ]:

# Collect all model results
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    loss = log_loss(y_test, y_prob)
    results[name] = {"accuracy": acc, "auc": auc, "loss": loss}

# Bar plots
names = list(results.keys())
accuracies = [results[n]['accuracy'] for n in names]
aucs = [results[n]['auc'] for n in names]
losses = [results[n]['loss'] for n in names]

plt.figure(figsize=(10,5))
sns.barplot(x=names, y=accuracies)
plt.xticks(rotation=45)
plt.title("Model Accuracy Comparison")
plt.ylabel("Accuracy")
plt.show()

plt.figure(figsize=(10,5))
sns.barplot(x=names, y=aucs)
plt.xticks(rotation=45)
plt.title("Model AUC Score Comparison")
plt.ylabel("AUC Score")
plt.show()

plt.figure(figsize=(10,5))
sns.barplot(x=names, y=losses)
plt.xticks(rotation=45)
plt.title("Model Log Loss Comparison")
plt.ylabel("Log Loss")
plt.show()
